EMOTION RECOGNITION

STEP: 
-Tokenizzazione
-Lemmatizzazione
-Stemming
-Rimuovere stop word
-Trasformare tutto in minuscolo
-far diventare 'not good' in 'not_good' (vedo dopo):/
-Rilevamento di Bigram (token che si verificano spesso in coincidenza) utilizzando 'Phrases' di gensim  :/

Funzioni Topic recognition

In [ ]:
def remove_link_menzione(doc):
  doc=re.sub(pattern_link, '',doc)
  doc=re.sub(pattern_menzione," ",doc)
  
  return doc

In [ ]:
def clear_corpus(corpus):
  my_not=set()
  tmp=''
  tokens_=[]
  lemmatized_string=list()
  post=list()


  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  wnl = WordNetLemmatizer()

  for frase in corpus:
    regex=re.compile("not \w+")
    for a in regex.findall(frase):
     my_not.add(a)
  


  tokens_ = [list(gensim.utils.tokenize(remove(doc), lower=True)) for doc in corpus]

  for lista in tokens_:
    #if len(lista)>0: #per togliermi quelle vuote che magari contenevano solo link
    parole_esaminate=[]
  
    for parole in lista:
      #if not parole in all_stopwords:
            if parole == 'not':
              tmp='not_'
            elif tmp=='not_' and parole not in all_stopwords:
                parole=tmp+parole
                #parole_esaminate.append(parole)
                tmp=''
            if parole != 'not' and parole not in all_stopwords:
              parole_esaminate.append(parole)

    lemmatized_string.append([wnl.lemmatize(words) for words in parole_esaminate ]) 

  bigram_mdl = gensim.models.phrases.Phrases(lemmatized_string, min_count=1, threshold=2)##farglielo fare su tutto il corpus tokenizzato altrimenti non funziona se lo fai su una sola frase
  for l in lemmatized_string:

    bigrams = bigram_mdl[l]
    post.append(bigrams)

  lemmatized_bigram_string=[' '.join(word) for word in post]
return lemmatized_bigram_string,post



In [ ]:
def tf_idfLSA(lemmatized_bigram_string):
  modello= TfidfVectorizer()
  M=modello.fit_transform(lemmatized_bigram_string)
  termini=modello.get_feature_names()

return modello,M,termini

In [ ]:
def lsa(M,termini):
  lsa = TruncatedSVD(n_components=10,n_iter=300)
  lsa.fit(M)

return lsa

In [ ]:
def print_concettiLSA(lsa):
   concetti={}
#usare questo quelli di prima sono prove
  for i,j in enumerate(lsa.components_):
    parolaChiave=zip(termini,abs(j))
    paroleOrdinate = sorted(parolaChiave,key=lambda x:x[1], reverse = True)
    paroleOrdinate = paroleOrdinate[:10]
    concetti["concetto_" + str(i)]=[]
    print("Asse concettuale n. ",i, ": ")
    for k in paroleOrdinate:
      concetti["concetto_" + str(i)].append(k)
      print(k)
    print('\n\n')
return concetti


In [ ]:
def punteggi_LSA(concetti,):
  punteggi=dict()

  for key,lista in concetti.items():
    punteggio=0
    for i in parole:
      for concetto_parola,valore in lista:
        if(i == concetto_parola):
          punteggio+=valore
    punteggi[key]=punteggio

return punteggi

In [ ]:
def result_TopicLSA(corpus,punteggi):
  risultato = []
  print("Risultato di ", corpus)
  for concetto,valore in punteggi.items():
    if valore!=0:
      print(concetto, concetti[concetto])

In [ ]:
def result_Topic_max(corpus,punteggi):
  risultato = []
  massimo = max(punteggi.values())
  print("Risultato di ", corpus)
  for concetto,valore in punteggi.items():
    if valore == massimo:
      risultato.append((concetto, concetti[concetto]))
  return risultato

In [ ]:
def lda(post):
  parole_valore=[]
  concetti_LDA={}

  dictionary = gensim.corpora.Dictionary(post)
  corpus2 = [dictionary.doc2bow(text) for text in post]

  tfidf = gensim.models.TfidfModel(corpus2)
  corpus_tfidf = tfidf[corpus2]

  ldamodel = gensim.models.ldamodel.LdaModel(corpus2, num_topics=10, id2word=dictionary, passes=500)
  

  for idx,lista_t in topics:
    parole_valore=[]
    tmp_p=re.sub('[\"]',"",lista_t)
    tmp_p=tmp_p.split(' + ') 
    coppia=list(t.split('*') for t in tmp_p)
    concetti_LDA['concetto_'+str(idx)]=[]
    for valore,parola in coppia:
      concetti_LDA['concetto_'+str(idx)].append((parola,valore))

return ldamodel,concetti_LDA

In [ ]:
def punteggioLDA(concetti_LDA,parole):
  punteggi=dict()
  for key in concetti_LDA.keys():
    punteggio=0
    for i in parole:
      for concetto_parola,valore in concetti_LDA[key]:
        if(i == concetto_parola):
         punteggio+=float(valore)
    punteggi[key]=punteggio
  return punteggi

In [ ]:
def result_TopicLDA(corpus,concetti_LDA):
  risultato = []
  p=punteggioLDA(concetti_LDA,parole)
  massimo = max(p.values())
  print("Risultato di ", corpus)
  for concetto,valore in p.items():
    if valore == massimo:
      risultato.append((concetto, concetti_LDA[concetto]))
  return (risultato)

In [ ]:
def tf_idfLDA(corpusLDA):
  tfidf = gensim.models.TfidfModel(corpusLDA)
  corpus_tfidf = tfidf[corpusLDA]

In [ ]:
def grafico_LDA(ldamodel,corpus2,dictionary):

  pyLDAvis.enable_notebook()
  lda_display = gensimvis.prepare(ldamodel, corpus2, dictionary, sort_topics=False)

  pyLDAvis.display(lda_display)

In [ ]:
def hdp(corpus2):
  hdpmodel = HdpModel(corpus=corpus2, id2word=dictionary)
  hdptopics = hdpmodel.show_topics(formatted=False)

In [ ]:
def punteggioHDP(concetti_HDP,parole):
  punteggi=dict()
  risultato = []
  for key,lista in concetti_HDP:
    punteggio=0
    for i in parole:
      for concetto_parola,valore in lista:
        if(i == concetto_parola):
         punteggio+=valore
    if punteggio!=0:
        risultato.append((key,lista))
    punteggi[key]=punteggio
  return punteggi

In [ ]:
def result_TopicHDP(concetti_HDP,parole):
  risultato = []
  p=punteggioHDP(concetti_HDP,parole)
  massimo = max(p.values())
  #print("Risultato di ", corpus)
  for concetto,valore in p.items():
    if valore == massimo:
      risultato.append((concetto, concetti_HDP[concetto]))
  return (risultato)

########################################################################################

In [ ]:
def clean_text(text):
  
  #pattern_simboli="[!\"£$%&/()=?^*+\[\]{}#§@><\\|,;:_]"  lascio # non so se togliere
  pattern_link="https\S+"
  #pattern_puntiCons="[\.]+"
  pattern_menzione="@\S+"
  #pattern_spazi="\s+"
  #dizionarioSentiment= dict()
   
  sp = spacy.load('en_core_web_sm')
  all_stopwords = sp.Defaults.stop_words
  wnl = WordNetLemmatizer()

  #content=text.lower()
  content=re.sub(pattern_link, '',text)
  content=re.sub(pattern_menzione," ",content)
  
  docsplit=list(gensim.utils.tokenize(content, lower=False))

  return docsplit

In [ ]:
prova=" assault rifle user friendly New York Medium blog assault riffle https://colab.research.google.com/drive/1vLSJBYqC8l4jtVueeNmyMhinRiVmS-tP#scrollTo=ji-puDKGhJks ciao come stai ... lol Trump FFRGGGDSS #jshdj @hsgays;  . :   not good, good  "

clean_text(prova)

In [ ]:
def txt_preprocessing(text):
  
    tmp=''
    lemmatized_string=[]
    parole_esaminate=[]
    dizionario=set()
    lst1 = list()
    lst2 = list()

    list_of_word=['impressed','sure','good','bad','happy','worry','able','fast','super','know','peaceful','cool','sad','confused','cool','attractive','gracious','sick','best','fan','care','love','sunny','funny','dead','annoying','easy','great','cold','regret','sick','like','hard','fun','enjoying','nice','bored','true','mad','angry','afraid']

    bigram_mdl = gensim.models.phrases.Phrases(text, min_count=1, threshold=2)

    sp = spacy.load('en_core_web_sm')
    all_stopwords = sp.Defaults.stop_words
    wnl = WordNetLemmatizer()
    
    #da fare ma meglio perchè facendolo l'accuratezza diminuiva
    for parole in text:
          if parole == 'not':
            tmp='not_'
          elif tmp=='not_' and len(parole)>2 and parole in list_of_word:
              parole=tmp+parole
              tmp=''
          if parole != 'not':
            parole_esaminate.append(parole)

    lemmatized_string.append([wnl.lemmatize(words) for words in parole_esaminate if not words in all_stopwords ])
    print(lemmatized_string)
    bigrams = bigram_mdl[lemmatized_string]
    pprint.pprint(list(bigrams))

    #dizionario.update(lemmatized_string)
    #lst2.append(lemmatized_string)

    return bigrams

In [ ]:
pprint.pprint(list(txt_preprocessing(clean_text(prova))))#non funziona bgram

In [ ]:
def choose_emotion_word_features(emoct1,emoct2,df):
    df2 = df[df['sentiment'].isin([emoct1, emoct2])]

#crea una lista di tuple che contengono sentiment e content
    documenti = list(zip(df2['content'], df2['sentiment']))
    all_words = nltk.FreqDist()


    for coppie in documenti:
      for parole in coppie[0]:
         all_words[parole] += 1
    
    word_features = list(all_words.keys())[:len(all_words)]

    return word_features


In [ ]:
def find_features(document,word_features):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

In [ ]:
def training_and_classifier(find_features,documenti):
  featuresets = [(find_features(rev), category) for (rev, category) in documenti]
  training_set = featuresets[:8000]#sistemare fare di più training e testing metà
  testing_set = featuresets[5000:]
  classifier = nltk.NaiveBayesClassifier.train(training_set)

  classifier.show_most_informative_features(10)
  print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

  return classifier

In [ ]:
def save_classifier(classifier, name):
  save_classifier = open(name+".pickle","wb")
  pickle.dump(classifier, save_classifier)
  save_classifier.close()


In [ ]:
def load_classifier(name):
  classifier_f = open(name+".pickle", "rb")
  classifier = pickle.load(classifier_f)
  classifier_f.close()

In [ ]:
def classifica(features):
        votes = []
        
        categoria = classifier.classify(features)
        votes.append(categoria)
        return mode(votes)

In [ ]:
def sentiment(text):
    feats={}
    feature_all = find_features(text)
    for word, value in feature_all.items():
      if value == True:
        feats[word]=value
        print(word, value)
    #print(feats)
    return classifica(feats)
    #,voted_classifier.confidence(feats)